In [ ]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import warnings 
warnings.filterwarnings("ignore")
Path.cwd()

## Test Data Generation

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import numpy as np
n_points = 10000
p, d, q = 5, 2, 5
model = ARIMA(np.random.randn(n_points), order=(p, d, q))
model_fit = model.fit()
water_demand = model_fit.predict(start=0, end=n_points-1)

In [ ]:
print(model_fit.summary())

In [ ]:
water_demand[100:].min(), water_demand[100:].max()

## genertate data 

- seed > 100  leak_increase_range=(5, 10)
- seed > 200 leak_increase_range=(4,8)

In [ ]:
# generate 10 independent water demands data 
# for seed in range(50):
for seed in range(242,247):
    num_leaks = 12
    df = wd.simulation(seed, num_leaks)
    wd.save(df, seed, num_leaks)

## Change leakage rate range

In [ ]:
## window size
dft =  wd.load(43, 12) 
ws = 10
dft['rolling_mean'] = dft['WaterDemandWithLeaks'].rolling(window=ws,center=True).mean()

dft2 = wd.load(243, 12)
dft2['rolling_mean'] = dft2['WaterDemandWithLeaks'].rolling(window=ws,center=True).mean()

import src.utils.plot_utils as pu
import matplotlib.pyplot as plt
pu.setup_mpl(as_default=0)

dfp = dft[1100:1200]
dfp2 = dft2[3880:3980]

fig,ax = plt.subplots(1,2, figsize=(3.60236*2, 3.5),dpi=300)

ax[0].plot(dfp.index, dfp['WaterDemandWithLeaks'],
            label='Water Demand with Leaks', alpha=0.75)
ax[0].scatter(dfp[dfp['LeakageLabel'] == 1].index, dfp[dfp['LeakageLabel']
            == 1]['WaterDemandWithLeaks'], color='red', label='Leakages',s=3)
ax[0].plot(dfp.index, dfp['rolling_mean'], label='Rolling Mean', color='black')
ax[0].legend()
ax[0].set_xlabel('Time')
ax[0].set_ylabel('Normalized Water Demand')
ax[0].set_title('Leakages Increasing Range from 0.5 to 1')

ax[1].plot(dfp2.index, dfp2['WaterDemandWithLeaks'],
            label='Water Demand with Leaks', alpha=0.75)
ax[1].scatter(dfp2[dfp2['LeakageLabel'] == 1].index, dfp2[dfp2['LeakageLabel']
            == 1]['WaterDemandWithLeaks'], color='red', label='Leakages',s=3)   
ax[1].plot(dfp2.index, dfp2['rolling_mean'], label='Rolling Mean', color='black')
ax[1].legend()

ax[1].set_xlabel('Time')
ax[1].set_ylabel('Normalized Water Demand')
ax[1].set_title('Leakages Increasing Range from 4 to 8')

for i in range(2):
    # label A B 
    ax[i].text(-0.1, 1.05, chr(65+i), transform=ax[i].transAxes,
            size=12, weight='bold')
fig.savefig('fig/leakages_increasing_range.png', bbox_inches='tight')